In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import LSTM, GRU
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from __future__ import print_function
from keras.layers.core import Activation, TimeDistributedDense, RepeatVector
from keras.layers import recurrent
import numpy as np

Using Theano backend.
Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, CuDNN 4007)


In [3]:
class CharacterTable(object):
    def __init__(self, vocab, maxlen):
        self.vocab = vocab
        self.maxlen = maxlen
    
    def encode(self, C, maxlen=None):
        maxlen = maxlen if maxlen else self.maxlen
        X = np.zeros((maxlen, len(self.vocab)))
        for i, c in enumerate(C):
            X[i, c] = 1
        return X
    
    def decode(self, X, calc_argmax=True):
        if calc_argmax:
            X = X.argmax(axis=-1)
        return ','.join(x for x in X)
    
def generateRandSeq(min, max, len):
    return [np.random.randint(min, max) for _ in range(len)]

In [41]:
TRAINING_SIZE = 150000
TEST_SIZE = 10000
DIGITS = 25
MAXLEN = DIGITS
voc = list(xrange(1000))
ctable = CharacterTable(voc, MAXLEN)

In [42]:
inputs = []
outputs = []
inputs_t = []
outputs_t = []
print('Generating data...')
while len(inputs) < TRAINING_SIZE:
    s = generateRandSeq(0, len(voc), DIGITS)
    inputs.append(s)
    s2 = [i%200 for i in s]
    outputs.append(sorted(s2)[::-1])

while len(inputs_t) < TEST_SIZE:
    s = generateRandSeq(0, len(voc), DIGITS)
    inputs_t.append(s)
    s2 = [i%200 for i in s]
    outputs_t.append(sorted(s2)[::-1])
print(inputs[12])
print(outputs[12])

Generating data...
[353, 115, 172, 440, 52, 411, 662, 130, 443, 675, 95, 744, 166, 31, 985, 198, 932, 467, 201, 371, 433, 675, 206, 677, 443]
[198, 185, 172, 171, 166, 153, 144, 132, 130, 115, 95, 77, 75, 75, 67, 62, 52, 43, 43, 40, 33, 31, 11, 6, 1]


In [43]:
print('Vectorization...')
X = np.zeros((len(inputs), MAXLEN), dtype=np.int32)
# y = np.zeros((len(outputs), MAXLEN), dtype=np.int32)
y = np.zeros((len(outputs), MAXLEN, len(voc)), dtype=np.bool)
for i, sentence in enumerate(inputs):
    X[i] = inputs[i]

# for i, sentence in enumerate(outputs):
#     y[i] = outputs[i]
for i, sentence in enumerate(outputs):
    y[i] = ctable.encode(sentence, maxlen=MAXLEN)

X_test = np.zeros((len(inputs_t), MAXLEN), dtype=np.int32)
# y_test = np.zeros((len(outputs_t), MAXLEN), dtype=np.int32)
y_test = np.zeros((len(outputs_t), MAXLEN, len(voc)), dtype=np.bool)
for i, sentence in enumerate(inputs_t):
    X_test[i] = inputs_t[i]

# for i, sentence in enumerate(outputs_t):
#     y_test[i] = outputs_t[i]
for i, sentence in enumerate(outputs_t):
    y_test[i] = ctable.encode(sentence, maxlen=MAXLEN)
    
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

Vectorization...
(150000, 25)
(150000, 25, 1000)
(10000, 25)
(10000, 25, 1000)


In [44]:
HIDDEN_SIZE = 256
BATCH_SIZE = 200
LAYERS = 2
'''
Hey guys, I also met this problem and I found this thread. Basically, 
the error info can happen when the dimension of the input data (X_train or Y_train) doesn't match with the 
model's input shape.

In my case (and @LeavesBreathe 's case I guess), the problem is that 
the model is expecting the Y_train to be a 3d tensor. Because of the embedding layer, 
the 2d tensor X_train of size (n_batch, sequence_length) will be eventually converted to a 3d tensor of size 
(n_batch, sequence_length, embedding_size) and will be processed by the succeeding LSTM layer. However, 
the 2d tensor Y_train of size (n_sample, sequence_length) is not converted to 3d, 
which is needed by the decoder LSTM.

To fix this problem, what I did is to convert Y_train into a 3d binary tensor (binary one-hot coding) and it worked.
'''
print('Build model...')
model = Sequential()
model.add(Embedding(len(voc), 300, input_length = MAXLEN))
model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
for _ in range(LAYERS - 2):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

model.add(LSTM(HIDDEN_SIZE))
model.add(RepeatVector(MAXLEN))
for _ in range(LAYERS):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

model.add(TimeDistributedDense(input_dim=HIDDEN_SIZE, output_dim=300))
model.add(Activation('tanh'))
model.add(TimeDistributedDense(input_dim=300, output_dim=len(voc)))
model.add(Activation('softmax'))

model.compile(optimizer='RMSprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Build model...


In [45]:
early_stopping = EarlyStopping(monitor='val_loss', patience=6)
hist = model.fit(X, y, batch_size=BATCH_SIZE, nb_epoch=150, 
                 callbacks=[early_stopping],
          validation_split = 0.1, shuffle=True)

score, acc = model.evaluate(X_test, y_test,
                            batch_size=BATCH_SIZE,
                            show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)

Train on 135000 samples, validate on 15000 samples
Epoch 1/150
135000/135000 [==============================] - 545s - loss: 4.1375 - acc: 0.0297 - val_loss: 3.9156 - val_acc: 0.0338
Epoch 2/150
135000/135000 [==============================] - 603s - loss: 3.7152 - acc: 0.0415 - val_loss: 3.5557 - val_acc: 0.0469
Epoch 3/150
135000/135000 [==============================] - 610s - loss: 3.4321 - acc: 0.0543 - val_loss: 3.3369 - val_acc: 0.0571
Epoch 4/150
135000/135000 [==============================] - 588s - loss: 3.2718 - acc: 0.0632 - val_loss: 3.1051 - val_acc: 0.0733
Epoch 5/150
135000/135000 [==============================] - 585s - loss: 3.0382 - acc: 0.0795 - val_loss: 2.9223 - val_acc: 0.0903
Epoch 6/150
135000/135000 [==============================] - 478s - loss: 2.8638 - acc: 0.0947 - val_loss: 2.8184 - val_acc: 0.0986
Epoch 7/150
135000/135000 [==============================] - 475s - loss: 2.7041 - acc: 0.1108 - val_loss: 2.6392 - val_acc: 0.1181
Epoch 8/150
135000/135000